**Tabular Playground Series is always a nice place to test new techniques and learn something new every month.**

Tried a different approach this month.

Still on 1.7511.

**Any help would be appreciated to help me reach a better score.**

Also, have done extensive EDA, will be uploading a full detailed EDA intuition and Inference Notebook pretty soon!

**Importing Relevant Libraries for Exploration, Plotting and use of Models through our analysis!**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import umap
import seaborn as sns

pd.options.display.max_columns = None
pd.options.display.max_rows = None

**Importing the dataset**

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-jun-2021/train.csv")

**Running some initial analysis!**

1. Seeing given data has 77 columns.

2. First column for Id Numbers.

3. Last column for Output Classes. Total of 9 classes.

4. 75 Feature columns ( Large number of features makes us think if feature engineering might be required).

5. 200k rows in the dataset.

6. No missing Values in dataset.

7. All values are integers.

8. **Max class frequency is of Class 6**

9. Maximum values of features are distributed between 0-2 though there are different maximums for different features.

10. Conclusion will be that this is a **Multi Class Classification problem** and we will treat it like that

In [ ]:
data.head()

In [ ]:
x = data.iloc[:,1:76]
y = data.iloc[:,-1]

x_predict_data = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
x_predict = x_predict_data.iloc[:,1:76]


In [ ]:
x.describe()

In [ ]:
y.describe()

In [ ]:
x.head()

In [ ]:
x.isna().sum()

In [ ]:
x.dtypes

In [ ]:
pd.crosstab(x.iloc[:,0],y)
#for i in range(0,75):
#   print(pd.crosstab(x.iloc[:,i],y))

**A major difference I noticed compared to last month's TPS was the presence of Duplicate values i.e.. Feature collection having same value but giving a different classification output.**

**Tried running the models with / without it. Got somewhat similar performance. Still have to deal with it.**

**330 values from a row count of 200k**

**It might just be a kind of trap in the data or the data might have been taken from a whole larger dataset / encoded from real world values which I was unable to figure out considering the data is numeric with no feature descriptions.**

In [ ]:
dup = x.loc[x.duplicated(keep=False)]
dup.shape

**Considering the Max, Min values and frequency distribution of values in all the features, We can see there is a presence of skewness with majority values lying between 0-2(for example). Hence Plotting a Box plot to check if skewness exists.**

In [ ]:
plt.figure(figsize = (76,6));
x1 = x.iloc[:,0:2]
sns.boxplot(x = 'variable', y = 'value',data =x1.melt()).set_title('Original Feature Box Plots');

**Using Label Encoder to encode output classes so its easier for our models to do predictions.**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

**Splitting the Dataset into train and test data.**

This allows us to split the current dataset and work on it to get the most optimum solution out of it, train our model and then apply it to predict probabilities on Predition(test) data.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test  = train_test_split(x,y,train_size=0.75,random_state=42)

**How to deal with Skewness in data?**

For skewness in data in Machine Learning problems the best way is to tackle it by standardizing data by applying Transformation, Scaling or both Transformation and Scaling.

**I have used Logarithmic Transformation in combination with Standard Scaler (between 0-1) to standardize my data.**

In [ ]:
x_train = x_train.applymap(lambda p: np.log(p+1))

x_test = x_test.applymap(lambda p: np.log(p+1))

x_predict = x_predict.applymap(lambda p: np.log(p+1))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
x_predict = sc.fit_transform(x_predict)

**Now the Question arises if we need to do Principal Component Analysis (PCA)?**

As the number of features are large, it is a quick thought to reduce dimentionality of the dataset and to use only those features which might affect our outcome.

I tried Plotting a graph of Cumulative variance vs No. of components

This is being done to use only the number of relevant component features which affect the outcome. Threshold being selected as 95%

But then I realise that it is a classification problem and 95% covers almost all of our features and we might miss out on important information about our output classes if we reduce any number of features.

Hence, i decided to skip PCA for this month's TPS solution

Please feel free to try this approach and let me know in comments if PCA gets us better outputs.



In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(x_train)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1, 76, step=1)
yi = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 75, step=2)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

**Applying Catboost to predict output probabilities**

In [ ]:
import lightgbm as lgb
params = {'bagging_freq': 1, 
          'verbosity': -1,  
          'num_threads': -1, 
          'feature_pre_filter': False,  
          'objective': 'multiclass', 
          'metric': 'multi_logloss',  
          'bagging_fraction': 0.5, 
          'feature_fraction': 0.8, 
          'lambda_l1': 10, 
          'lambda_l2': 10, 
          'learning_rate': 0.013959172480364537, 
          'max_depth': 6, 
          'min_child_samples': 25, 
          'num_leaves': 31}
model = lgb.LGBMClassifier(**params)
model.fit(x_train,y_train)

In [ ]:
#from catboost import CatBoostClassifier
#model = CatBoostClassifier(iterations = 4000,reg_lambda=100,learning_rate = 0.01,bootstrap_type='Bernoulli',random_strength = 5,depth = 8,task_type = 'GPU',loss_function='MultiClass',silent = True)
#model.fit(x_train,y_train)

In [ ]:
y_pred_proba = model.predict_proba(x_predict)

In [ ]:
output = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")
output_df = pd.DataFrame(y_pred_proba, columns=['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9'])
output_df = pd.concat([x_predict_data.id,output_df],axis = 1)
output_df.head()

In [ ]:
output_df.to_csv('./Output_Catb.csv', index=False)